# 제품 이상여부 판별 프로젝트


### 데이터 읽어오기


In [1]:
import os
from pprint import pprint

import optuna
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
import pandas as pd

RANDOM_STATE = 110

train_data = pd.read_csv('./data/train_data_0827.csv')
test_data = pd.read_csv('./data/test_data_0827.csv')

In [3]:
# dam, fill1, fill2 공통 변수
var_dam_fill = [
    'Receip_No_encoded',
    'Equipment_same_num',
    'PalletID_Collect_Result_encoded',
    'Production_Qty_Collect_Result',
    'WorkMode Collect Result'
]

In [4]:
# 전체 공통 변수
### train
var_all_train = [
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

### test
var_all_test = [
    'Set ID',
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

In [5]:
# '_Dam'을 포함하는 변수 선택
dam_variables = [var for var in train_data.columns if '_Dam' in var]

# train
final_columns_train = var_dam_fill + var_all_train + dam_variables
train_data_dam = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + dam_variables
test_data_dam = test_data[final_columns_test]

In [6]:
# '_Fill1'을 포함하는 변수 선택
fill1_variables = [var for var in train_data.columns if '_Fill1' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill1_variables
train_data_fill1 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill1_variables
test_data_fill1 = test_data[final_columns_test]

In [7]:
# '_Fill2'을 포함하는 변수 선택
fill2_variables = [var for var in train_data.columns if '_Fill2' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill2_variables
train_data_fill2 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill2_variables
test_data_fill2 = test_data[final_columns_test]

In [8]:
# '_AutoClave'을 포함하는 변수 선택
autoclave_variables = [var for var in train_data.columns if '_AutoClave' in var]

# train
final_columns_train = var_all_train + autoclave_variables
train_data_autoclave = train_data[final_columns_train]

# test 
final_columns_test = var_all_test + autoclave_variables
test_data_autoclave = test_data[final_columns_test]

In [9]:
# 각 DataFrame의 칼럼 수 계산
num_columns_train_data = train_data.shape[1]
num_columns_train_data_dam = train_data_dam.shape[1]
num_columns_train_data_autoclave = train_data_autoclave.shape[1]
num_columns_train_data_fill1 = train_data_fill1.shape[1]
num_columns_train_data_fill2 = train_data_fill2.shape[1]

num_columns_test_data = test_data.shape[1]
num_columns_test_data_dam = test_data_dam.shape[1]
num_columns_test_data_autoclave = test_data_autoclave.shape[1]
num_columns_test_data_fill1 = test_data_fill1.shape[1]
num_columns_test_data_fill2 = test_data_fill2.shape[1]

# 각 DataFrame의 칼럼 수 출력
print("----train data-----")
print(f"train_data DataFrame의 칼럼 수: {num_columns_train_data}")
print(f"train_data_dam DataFrame의 칼럼 수: {num_columns_train_data_dam}")
print(f"train_data_autoclave DataFrame의 칼럼 수: {num_columns_train_data_autoclave}")
print(f"train_data_fill1 DataFrame의 칼럼 수: {num_columns_train_data_fill1}")
print(f"train_data_fill2 DataFrame의 칼럼 수: {num_columns_train_data_fill2}")
print("----test data-----")
print(f"test_data DataFrame의 칼럼 수: {num_columns_test_data}")
print(f"test_data_dam DataFrame의 칼럼 수: {num_columns_test_data_dam}")
print(f"test_data_autoclave DataFrame의 칼럼 수: {num_columns_test_data_autoclave}")
print(f"test_data_fill1 DataFrame의 칼럼 수: {num_columns_test_data_fill1}")
print(f"test_data_fill2 DataFrame의 칼럼 수: {num_columns_test_data_fill2}")

----train data-----
train_data DataFrame의 칼럼 수: 40
train_data_dam DataFrame의 칼럼 수: 23
train_data_autoclave DataFrame의 칼럼 수: 8
train_data_fill1 DataFrame의 칼럼 수: 14
train_data_fill2 DataFrame의 칼럼 수: 14
----test data-----
test_data DataFrame의 칼럼 수: 41
test_data_dam DataFrame의 칼럼 수: 24
test_data_autoclave DataFrame의 칼럼 수: 9
test_data_fill1 DataFrame의 칼럼 수: 15
test_data_fill2 DataFrame의 칼럼 수: 15


---

## Optuna

스레스홀드 0.3으로 맞춘상태에서 튜닝 진행한 것

In [10]:
RANDOM_STATE = 110
THRESHOLD = 0.3

In [ ]:
# 'Normal'과 'AbNormal'을 숫자로 변환
train_data_fill2['target'] = train_data_fill2['target'].map({'Normal': 0, 'AbNormal': 1})

# 스레드홀드 설정
THRESHOLD = 0.3

def objectiveRandomForestClassifier(trial, x_tr, y_tr, x_val, y_val):
    param = {
        'n_estimators' : trial.suggest_int('n_estimators', 800, 3000),
        'max_depth' : trial.suggest_int('max_depth', 20, 100),
        'min_samples_split' : trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf' : trial.suggest_int('min_samples_leaf', 1, 8),
        'criterion' : trial.suggest_categorical("criterion", ["entropy",]),
        'class_weight' : trial.suggest_categorical("class_weight", ["balanced"]),
        'random_state': RANDOM_STATE
    }
    
    model = RandomForestClassifier(**param)
    model.fit(x_tr, y_tr)
    pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    pred = (pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측
    
    score = f1_score(y_val, pred, average="binary")
    
    return score

# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    train_data_fill2.drop("target", axis=1),
    train_data_fill2["target"],
    test_size=0.2,
    shuffle=True,
    random_state=RANDOM_STATE,
)

# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
study.optimize(lambda trial: objectiveRandomForestClassifier(trial, x_train, y_train, x_val, y_val), n_trials=300)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

/tmp/ipykernel_284/4128727499.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_fill2['target'] = train_data_fill2['target'].map({'Normal': 0, 'AbNormal': 1})
[I 2024-08-28 08:54:42,263] A new study created in memory with name: no-name-1b4ca16c-cf84-4997-886b-5221a13579f6
[I 2024-08-28 08:55:03,760] Trial 0 finished with value: 0.17783857729138167 and parameters: {'n_estimators': 1055, 'max_depth': 73, 'min_samples_split': 9, 'min_samples_leaf': 5, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 0 with value: 0.17783857729138167.
[I 2024-08-28 08:55:48,524] Trial 1 finished with value: 0.16624113475177307 and parameters: {'n_estimators': 2295, 'max_depth': 75, 'min_samples_split': 20, 'min_samples_leaf': 6, 'criter

[I 2024-08-28 09:13:28,602] Trial 28 finished with value: 0.18289085545722714 and parameters: {'n_estimators': 2525, 'max_depth': 78, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 23 with value: 0.19090098126672617.
[I 2024-08-28 09:13:47,309] Trial 29 finished with value: 0.17701543739279588 and parameters: {'n_estimators': 921, 'max_depth': 70, 'min_samples_split': 10, 'min_samples_leaf': 5, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 23 with value: 0.19090098126672617.
[I 2024-08-28 09:14:22,888] Trial 30 finished with value: 0.17345679012345677 and parameters: {'n_estimators': 1785, 'max_depth': 94, 'min_samples_split': 7, 'min_samples_leaf': 6, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 23 with value: 0.19090098126672617.
[I 2024-08-28 09:15:27,652] Trial 31 finished with value: 0.18091451292246521 and parameters: {'n_estimators': 2767, 'max_depth': 96, 'min_samples_s

[I 2024-08-28 09:38:53,967] Trial 57 finished with value: 0.18270008084074377 and parameters: {'n_estimators': 2500, 'max_depth': 43, 'min_samples_split': 6, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 43 with value: 0.19124218051831995.
[I 2024-08-28 09:39:46,107] Trial 58 finished with value: 0.17767653758542143 and parameters: {'n_estimators': 2392, 'max_depth': 55, 'min_samples_split': 8, 'min_samples_leaf': 2, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 43 with value: 0.19124218051831995.
[I 2024-08-28 09:40:31,497] Trial 59 finished with value: 0.17220338983050848 and parameters: {'n_estimators': 2257, 'max_depth': 20, 'min_samples_split': 3, 'min_samples_leaf': 3, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 43 with value: 0.19124218051831995.
[I 2024-08-28 09:41:40,748] Trial 60 finished with value: 0.1910714285714286 and parameters: {'n_estimators': 3000, 'max_depth': 50, 'min_samples_sp

[I 2024-08-28 10:09:20,314] Trial 86 finished with value: 0.1683366733466934 and parameters: {'n_estimators': 2666, 'max_depth': 39, 'min_samples_split': 5, 'min_samples_leaf': 7, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 43 with value: 0.19124218051831995.
[I 2024-08-28 10:10:18,492] Trial 87 finished with value: 0.18556701030927836 and parameters: {'n_estimators': 2561, 'max_depth': 48, 'min_samples_split': 7, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 43 with value: 0.19124218051831995.
[I 2024-08-28 10:11:23,704] Trial 88 finished with value: 0.18298144593730006 and parameters: {'n_estimators': 2952, 'max_depth': 66, 'min_samples_split': 6, 'min_samples_leaf': 2, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 43 with value: 0.19124218051831995.
[I 2024-08-28 10:12:28,449] Trial 89 finished with value: 0.17874875868917572 and parameters: {'n_estimators': 2752, 'max_depth': 59, 'min_samples_sp

[I 2024-08-28 10:41:05,299] Trial 115 finished with value: 0.17982017982017984 and parameters: {'n_estimators': 2891, 'max_depth': 36, 'min_samples_split': 4, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 43 with value: 0.19124218051831995.
[I 2024-08-28 10:42:08,064] Trial 116 finished with value: 0.18681318681318684 and parameters: {'n_estimators': 2825, 'max_depth': 58, 'min_samples_split': 5, 'min_samples_leaf': 2, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 43 with value: 0.19124218051831995.
[I 2024-08-28 10:43:12,423] Trial 117 finished with value: 0.16864175022789424 and parameters: {'n_estimators': 2954, 'max_depth': 47, 'min_samples_split': 15, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 43 with value: 0.19124218051831995.
[I 2024-08-28 10:44:21,727] Trial 118 finished with value: 0.19090098126672617 and parameters: {'n_estimators': 2989, 'max_depth': 41, 'min_samp

[I 2024-08-28 11:10:03,829] Trial 144 finished with value: 0.182 and parameters: {'n_estimators': 2513, 'max_depth': 56, 'min_samples_split': 4, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 43 with value: 0.19124218051831995.
[I 2024-08-28 11:11:04,416] Trial 145 finished with value: 0.18314424635332252 and parameters: {'n_estimators': 2649, 'max_depth': 53, 'min_samples_split': 6, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 43 with value: 0.19124218051831995.
[I 2024-08-28 11:11:59,358] Trial 146 finished with value: 0.18446601941747573 and parameters: {'n_estimators': 2420, 'max_depth': 97, 'min_samples_split': 6, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 43 with value: 0.19124218051831995.
[I 2024-08-28 11:12:56,834] Trial 147 finished with value: 0.19090098126672617 and parameters: {'n_estimators': 2511, 'max_depth': 51, 'min_samples_split': 5, 

[I 2024-08-28 11:38:25,168] Trial 173 finished with value: 0.19090098126672617 and parameters: {'n_estimators': 2478, 'max_depth': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 155 with value: 0.19285714285714284.
[I 2024-08-28 11:39:20,169] Trial 174 finished with value: 0.1843168957154406 and parameters: {'n_estimators': 2402, 'max_depth': 74, 'min_samples_split': 6, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 155 with value: 0.19285714285714284.
[I 2024-08-28 11:40:19,634] Trial 175 finished with value: 0.19141323792486584 and parameters: {'n_estimators': 2573, 'max_depth': 96, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 155 with value: 0.19285714285714284.
[I 2024-08-28 11:41:19,237] Trial 176 finished with value: 0.18181818181818182 and parameters: {'n_estimators': 2557, 'max_depth': 96, 'min_sa

[I 2024-08-28 12:06:29,394] Trial 202 finished with value: 0.19285714285714284 and parameters: {'n_estimators': 2548, 'max_depth': 99, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 201 with value: 0.19320214669051877.
[I 2024-08-28 12:07:28,007] Trial 203 finished with value: 0.19158460161145927 and parameters: {'n_estimators': 2552, 'max_depth': 98, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 201 with value: 0.19320214669051877.
[I 2024-08-28 12:08:27,131] Trial 204 finished with value: 0.19141323792486584 and parameters: {'n_estimators': 2574, 'max_depth': 97, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 201 with value: 0.19320214669051877.
[I 2024-08-28 12:09:26,278] Trial 205 finished with value: 0.19141323792486584 and parameters: {'n_estimators': 2573, 'max_depth': 97, 'min_sa

[I 2024-08-28 12:35:20,908] Trial 231 finished with value: 0.19124218051831995 and parameters: {'n_estimators': 2622, 'max_depth': 97, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 201 with value: 0.19320214669051877.
[I 2024-08-28 12:36:18,130] Trial 232 finished with value: 0.19158460161145927 and parameters: {'n_estimators': 2534, 'max_depth': 96, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 201 with value: 0.19320214669051877.
[I 2024-08-28 12:37:14,801] Trial 233 finished with value: 0.19302949061662197 and parameters: {'n_estimators': 2538, 'max_depth': 96, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 201 with value: 0.19320214669051877.
[W 2024-08-28 12:38:02,527] Trial 234 failed with parameters: {'n_estimators': 2519, 'max_depth': 92, 'min_samples_split': 5, 'min_samples_lea

[I 2024-08-28 12:05:31,027] Trial 201 finished with value: 0.19320214669051877 and parameters: {'n_estimators': 2535, 'max_depth': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 201 with value: 0.19320214669051877.